In [31]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.random import sample_without_replacement
import os
os.chdir("D:\Kaggle\Santander")
#print(os.listdir("../input"))
import sys

In [21]:
sys.stdout.write('Let me do this real quick! \n')
train_df = pd.read_csv('train.csv')
#train_Idx=sample_without_replacement(train.shape[0],20000)
#train_df=train.loc[train_Idx,:]

Let me do this real quick! 


In [22]:
test = pd.read_csv('test.csv') 
test_Idx=sample_without_replacement(test.shape[0],10000)
test_df=test.loc[test_Idx,:]

In [23]:
varimp=pd.read_csv('impvar.csv')

In [24]:
#features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
features = varimp.variable[0:150]
target = train_df['target']
param = {
    'bagging_freq': 5,          'bagging_fraction': 0.38,   'boost_from_average':'false',   'boost': 'rf',
    'feature_fraction': 0.045,   'learning_rate': 0.01,     'max_depth': -1,                'metric':'auc',
    'min_data_in_leaf': 80,     'min_sum_hessian_in_leaf': 10.0,'num_leaves': 13,           'num_threads': 8,
    'tree_learner': 'serial',   'objective': 'binary',      'verbosity': 1
}

folds = StratifiedKFold(n_splits=2, shuffle=False, random_state=44000)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))

In [25]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold :{}".format(fold_ + 1))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    clf = lgb.train(param, trn_data, 100000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 1000)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
sys.stdout.write("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = predictions
sub.to_csv('submission.csv', index=False)

Fold :1
Training until validation scores don't improve for 1000 rounds.
[1000]	training's auc: 0.903908	valid_1's auc: 0.877582
[2000]	training's auc: 0.906179	valid_1's auc: 0.879468
[3000]	training's auc: 0.906979	valid_1's auc: 0.880184
[4000]	training's auc: 0.907457	valid_1's auc: 0.880633
[5000]	training's auc: 0.907521	valid_1's auc: 0.880727
[6000]	training's auc: 0.907648	valid_1's auc: 0.880837
[7000]	training's auc: 0.908025	valid_1's auc: 0.881088
[8000]	training's auc: 0.908138	valid_1's auc: 0.881233
Early stopping, best iteration is:
[7745]	training's auc: 0.908194	valid_1's auc: 0.881287
Fold :2
Training until validation scores don't improve for 1000 rounds.
[1000]	training's auc: 0.900593	valid_1's auc: 0.881014
[2000]	training's auc: 0.902787	valid_1's auc: 0.883179
[3000]	training's auc: 0.903397	valid_1's auc: 0.884009
[4000]	training's auc: 0.903737	valid_1's auc: 0.88448
Early stopping, best iteration is:
[3952]	training's auc: 0.903774	valid_1's auc: 0.884554
CV 

In [30]:
#save Model
clf.save_model('lcf_model.txt')

In [32]:
#load model
clf = lgb.Booster(model_file='lcf_model.txt')